In [2]:
import random
import sqlite3
import pytz
import pandas as pd

from datetime import datetime, timedelta

import sys
sys.path.insert(0, "/Users/connorparish/code/hindsight")

from hindsight_server.utils import ocr_results_to_str
from hindsight_server.config import LLM_MODEL_NAME
from hindsight_server.query.query import load, llm_generate

/Users/connorparish/miniconda3/envs/hindsight_server/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
rem_db_path = "/Users/connorparish/Library/Containers/today.jason.rem/Data/Library/Application Support/today.jason.rem/db.sqlite3"
conn = sqlite3.connect(rem_db_path)

In [4]:
def preprocess_time_range(time_range):
    utc_time_range = list()
    for i, time in enumerate(time_range):
        alter_time = time.astimezone(pytz.UTC)
        utc_time_range.append(alter_time.strftime('%Y-%m-%dT%H:%M:%S.%f'))
    return utc_time_range

yesterday = datetime.now() - timedelta(days=1)
time_range = [yesterday, datetime.now()]
utc_time_range = preprocess_time_range(time_range)

In [5]:
query = f"""SELECT f.id as frameId, f.timestamp, f.activeApplicationName, ft.id as textId, ft.text, ft.x, ft.y, ft.w, ft.h FROM frames as f JOIN frames_text as ft ON f.id = ft.frameId WHERE f.timestamp BETWEEN '{utc_time_range[0]}' AND '{utc_time_range[1]}'"""

In [6]:
today_frames = pd.read_sql(query, conn)

In [7]:
all_frame_ids = set(today_frames['frameId'])
random_frame_ids = random.sample(list(all_frame_ids), 20)

In [8]:
frames = today_frames.loc[today_frames['frameId'].isin(random_frame_ids)]
frames['conf'] = 1

/var/folders/c_/2c9vmfhd35bgwc_6h0q80d180000gn/T/ipykernel_60584/3811660279.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frames['conf'] = 1


In [9]:
all_text = ""
for frame_id in frames['frameId'].unique():
    frame_df = frames.loc[frames['frameId'] == frame_id]
    all_text += ocr_results_to_str(frame_df)

In [10]:
prompt = f"""
You are an assistant who generates search queries for an embedding search database to help a user find content of interest.

Below is content that has been on the user's screen recently:

{all_text}

Task:
    - Analyze the content above.
    - Identify the main themes and topics.
    - Generate a single, concise sentence that summarizes the user's interests.
    - Include as many relevant keywords and concepts from the content as possible.
    - The sentence should be suitable as a search query for finding similar content. 

Return a JSON object with the following structure:
{{
    "query": "sentence for querying the embedding search database",
    "highlights": "Quick list of highlights used to generate the query",
    "quality": "float between 0 and 1 estimating the quality of the generated query for finding content of interest"
}}

Rules:
- Do not add backticks to the JSON eg \`\`\`json\`\`\` is WRONG
- DO NOT RETURN ANYTHING BUT JSON. NO COMMENTS BELOW THE JSON.
"""

# Pass to LLM

In [11]:
import mlx.core as mx
# from mlx_lm import load
from outlines import generate, models

In [12]:
# LLM_MODEL_NAME = "mlx-community/Llama-3.1-8B-Instruct"
# LLM_MODEL_NAME = "mlx-community/Meta-Llama-3-8B-Instruct-8bit"
LLM_MODEL_NAME = "mlx-community/Llama-3.2-3B-Instruct"

In [13]:
# Define JSON schema
json_schema = """
    {
    "type": "object",
    "properties": {
        "query": {"type": "string"},
        "highlights": {"type": "string"},
        "quality" : {"type" : "number"}
    }
}
"""

# Load MLX model
# model, tokenizer = load(LLM_MODEL_NAME)

# Create Outlines model wrapper
mlx_model = models.mlxlm(LLM_MODEL_NAME)


Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 124936.71it/s]


In [14]:
# model = models.mlxlm("mlx-community/Meta-Llama-3.1-8B-Instruct-8bit")
# generator = generate.text(model)

In [15]:
# answer = generator("My name is Connor Parish and ")

In [16]:
# Generate JSON output
generator = generate.json(mlx_model, json_schema)

In [20]:
output = generator(prompt)

IndexError: list index out of range

In [18]:
output

{'query': '', 'highlights': '', 'quality': 0.0}

In [19]:
prompt = """You are an assistant who generates search queries for an embedding search database to help a user find content of interest.

    Below is content that has been on the user's screen recently:

    Lock this man up immediately
Safest lock ever
GTA me trying to get into Los Santos Customs in
That BMW driver needs to be locked up for a long time
Grey SUV was not dealing with that
my uber driver whenever I want to be left alone in silence
We got extendo midgets before GTA 6 non aesthetic things Before and after limb lengthening @Pict... • 2d
SOMEONE HOPPED IN THE DRIVERS SEAT OF KAI CENAT U-HAUL AND DESTROYED HIS WHOLE SET UP GE
I'm glad my taxi driver has already been given the finger 3 times in 5 minutes
I'm glad my taxi driver has already been given the finger 3 times in 5 minutes 02

    Task:
        - Analyze the content above.
        - Identify the main themes and topics.
        - Generate a single, concise sentence that summarizes the user's interests.
        - Include as many relevant keywords and concepts from the content as possible.
        - The sentence should be suitable as a search query for finding similar content. 

    Return a JSON object with the following structure:
    {
        "query": "sentence for querying the embedding search database",
        "highlights": "Quick list of highlights used to generate the query",
        "quality": "float between 0 and 1 estimating the quality of the generated query for finding content of interest"
    }

    Rules:
    - Do not add backticks to the JSON eg \`\`\`json\`\`\` is WRONG
    - DO NOT RETURN ANYTHING BUT JSON. NO COMMENTS BELOW THE JSON."""